**3-site infinite time-evolving block decimation(iTEBD)**

This task considers an infinite 1D spin (ising - cluster) model (with PBC):
$$
H  = -\sum_j (gZ_{j-1} Y_j Z_{j+1} +JZ_j Z_{j+1} + hX_j),
$$
where $X$, $Y$ and $Z$ are Pauli matrices. 

We will calculate the ground-state energy, the magnetization per site $\langle X_i\rangle$ and $\langle Z_i\rangle$ and also entanglement entropy and spectrum of this model, using 3-site iTEBD method.

In [97]:
import numpy as np
import numpy.linalg as LA
import scipy.sparse.linalg as LAs
from scipy import linalg
import Sub180221 as Sub
import math,copy,pickle

In [107]:
Dp = 2
g = 1.0
J = 1.0
h = 1.0
Ds = 6

pZ = np.array([[1,0],[0,-1]])
pX = np.array([[0,1],[1,0]])
pY = np.array([[0,-1j],[1j,0]])
Id = np.eye(2)

In [108]:
def GetHam_IsingCluster(g,J,h):
    Ham = - g * np.kron(pZ,np.kron(pY,pZ)) - J * np.kron(Id, np.kron(pZ, pZ)) - h * np.kron(Id, np.kron(pX, Id))
    
    # reshape the 3-body Hamiltonian into a 6 - order tensor: 
    Ham = np.reshape(Ham,[Dp,Dp,Dp,Dp,Dp,Dp])
    return Ham

def GetExpHam(Ham,Tau):
    Dp = np.shape(Ham)[0]
    
    if LA.norm(Ham) < 1.0e-12:
        UH = np.reshape(np.eye(Dp**3),[Dp,Dp,Dp,Dp,Dp,Dp])
    else:
        # reshape hamiltonian tensor (6-order) into a matrix of size Dp**3, then apply eigenvalue decomposition
        A = np.reshape(Ham,[Dp**3,Dp**3])
        
        # Dc is the bond dimension of the diagonal matrix
        V,S,Dc = Sub.SplitEigh(A,Dp**3)
        
        # calculate e^{-\tau S}
        W = np.diag(np.exp(-Tau*S))
        
        # update e^A as new A
        A = np.dot(np.dot(V,W),np.transpose(np.conj(V)))
        
        # reshape UH = e^(-\tau H) (imaginary time evolution operator) into a 6-order tensor
        UH = np.reshape(A,[Dp,Dp,Dp,Dp,Dp,Dp])

    return UH

# initialize state randomly
def init_TG(Dp, Ds, n_sites):
    T = [None] * n_sites
    G = [None] * n_sites
    
    for i in range(n_sites):
        T[i] = np.random.rand(Ds, Dp ,Ds)
        G[i] = np.random.rand(Ds)

    return T,G

# evolution 2 bonds  - Gl - T1 - G1 - T2 - G2 - T3 - Gr - at a time in 3-site iTEBD
def Evo_2_bonds(Gl, T1, G1, T2, G2, T3, Gr, UH):
    A = Sub.NCon(
        [np.diag(Gl), T1, np.diag(G1), T2, np.diag(G2), T3, np.diag(Gr), UH],
        [[-1, 1], [1, 7, 2], [2, 3], [3, 8, 4], [4, 5], [5, 9, 6], [6, -5], [-2, -3, -4, 7, 8, 9]]
    )
    # record the shape of tensor A
    DA = np.shape(A)
    
    # get the matrization of tensor A, preparing for T1 and G1 updating
    matrix_A = Sub.Group(A, [[0,1],[2,3,4]])
    
    # update T1 and G1 using SVD decomp.
    U1, S1, V1 = np.linalg.svd(matrix_A, full_matrices=False)
    Dc1 = min(len(S1), Ds) #truncate SVD w.r.t. Ds
    U1 = U1[:, :Dc1]
    S1 = S1[:Dc1]
    V1 = V1[:Dc1, :]
    U1 = np.reshape(U1,[DA[0],DA[1],Dc1]) # reshape matrix U1 to component tensor shape (three legs)
    T1_new = np.tensordot(np.diag(1.0/Gl),U1,(1,0)) 
    
    # update T2 and G2 using SVD decomp.
    U2, S2, V2 = np.linalg.svd(np.reshape(np.diag(S1)@V1, [Ds * DA[2], -1]), full_matrices=False)
    Dc2 = min(len(S2), Ds) #truncate SVD w.r.t. Ds
    U2 = U2[:, :Dc2]
    S2 = S2[:Dc2]
    V2 = V2[:Dc2, :]
    T2_new = np.tensordot(np.diag(1/S1), np.reshape(U2, [Dc1, DA[2], Dc2]), (1,0))
    
    # update T3
    T3_new = np.tensordot(np.reshape(V2, [Dc2, DA[3], DA[4]]), np.diag(1.0 /Gr), (2, 0))

    G1_new = S1
    S1 /= np.sqrt(np.sum(S1 ** 2))
    G2_new = S2
    S2 /= np.sqrt(np.sum(S2 ** 2))
    
    return T1_new, G1_new, T2_new, G2_new, T3_new

# iterative itebd procedure:
def Evo_3site(Ds,Ham,Tau_list,Iter,Prec):
    Dp = np.shape(Ham)[0]
    T,G = init_TG(Dp, Ds, 3)

    r0 = 0
    for idt in range(len(Tau_list)):
        dt = Tau_list[idt]
        UH = GetExpHam(Ham,dt)

        G0 = np.ones(3)
        for r in range(Iter):
            for bond in range(3):
                T[bond], G[bond], T[(bond+1)%3], G[(bond+1)%3], T[(bond+2)%3] = Evo_2_bonds(
                    G[(bond-1)%3], T[bond], G[bond], T[(bond+1)%3], G[(bond+1)%3], T[(bond+2)%3], G[(bond+2)%3], UH
                )

            Err = 0.0
            for i in range(3):
                Err += np.abs(G[i][0]-G0[i])
            #if np.mod(r,100) == 1:
                #print(r+r0,Err)
            if Err < Prec[idt]:
                r0 += r
                break
            for i in range(3):
                G0[i] = G[i][0]
    print("Convergence is achieved!")
    return T,G

# calculate 3-body operator
def Cal_2_bonds(Op, Gl, T1, G1, T2, G2, T3, Gr):
    vec = Sub.NCon([np.diag(Gl), T1, np.diag(G1), T2, np.diag(G2), T3, np.diag(Gr)],
    [[-1, 1],[1, -2, 2], [2, 3], [3, -3, 4], [4, 5], [5, -4, 6], [6, -5]])
    expectation = Sub.NCon([vec, Op, np.conj(vec)],
    [[7, 1, 2, 3, 8], [4, 5, 6, 1, 2, 3], [7, 4, 5, 6, 8]])
    return expectation

# calculate energy after convergence is reached
def Cal_energy(T, G, Ham):
    D = np.shape(Ham)[0]
    
    # identity tensor for <psi|psi> calculation
    H00 = np.reshape(np.eye(D**3,D**3),[D,D,D,D,D,D])
    
    normalize = np.zeros(3)
    energy = np.zeros(3)
    
    for bond in range(3):
        
        normalize[bond] = np.real(\
        Cal_2_bonds(H00, G[(bond-1)%3], T[bond], G[bond], T[(bond+1)%3], G[(bond+1)%3], T[(bond+2)%3], G[(bond+2)%3]))
        
        energy[bond] = np.real(\
        Cal_2_bonds(Ham, G[(bond-1)%3], T[bond], G[bond], T[(bond+1)%3], G[(bond+1)%3], T[(bond+2)%3], G[(bond+2)%3]))
        
        energy[bond] /= normalize[bond]
        print(f'bond i = {bond}, energy: {energy[bond]}')
    
    energy = np.mean(energy)
    print(f'average energy: {energy}')
    
    return energy

# calculate x- and z- magnetization after convergence is reached
def Cal_mag(T, G):
    S0,Sp,Sm,Sz,Sx,Sy = Sub.SpinOper(Dp)
    D = Dp
    
    H00 = np.reshape(np.eye(D**3,D**3),[D,D,D,D,D,D])
    xmag_op = np.reshape(np.kron(pX, np.kron(Id, Id)),[D,D,D,D,D,D])
    zmag_op = np.reshape(np.kron(pZ, np.kron(Id, Id)),[D,D,D,D,D,D])
    
    xmag_val = np.zeros(3)
    zmag_val = np.zeros(3)
    normalize = np.zeros(3)
    
    for bond in range(3):
        normalize[bond] = np.real(\
        Cal_2_bonds(H00, G[(bond-1)%3], T[bond], G[bond], T[(bond+1)%3], G[(bond+1)%3], T[(bond+2)%3], G[(bond+2)%3]))
        xmag_val[bond] = np.real(\
        Cal_2_bonds(xmag_op, G[(bond-1)%3], T[bond], G[bond], T[(bond+1)%3], G[(bond+1)%3], T[(bond+2)%3], G[(bond+2)%3]))
        xmag_val[bond] /= normalize[bond]
        zmag_val[bond] = np.real(\
        Cal_2_bonds(zmag_op, G[(bond-1)%3], T[bond], G[bond], T[(bond+1)%3], G[(bond+1)%3], T[(bond+2)%3], G[(bond+2)%3]))
        zmag_val[bond] /= normalize[bond]
        print(f'bond i = {bond}, <\sigma_x>: {xmag_val[bond]}, <\sigma_z>: {zmag_val[bond]}')
    
    xmag_val = np.mean(xmag_val)
    zmag_val = np.mean(zmag_val)
    print(f'average <\sigma_x>: {xmag_val}')
    print(f'average <\sigma_z>: {zmag_val}')
    return xmag_val, zmag_val


# calculate entanglement entropy and spectrum after convergence is reached
def Cal_entanglement(T, G):
    entropy = np.zeros(3)
    spectrum = []
    for bond in range(3):
        entropy[bond] = -np.sum((G[bond]**2) * np.log((G[bond]**2)))
        spectrum.append(-np.log(G[bond]))
        print(f'bond i = {bond}, entanglement entopy: {entropy[bond]}, entanglement spectrum: {spectrum[bond]}')
    
    entropy = np.mean(entropy)
    spectrum = [(spectrum[0][i]+spectrum[1][i]+spectrum[2][i])/3 for i in range(len(spectrum[0]))]
    print(f'average entanglement entropy: {entropy}, average entanglement spectrum: {spectrum}')
    return None

In [113]:
Tau_list = [0.1, 0.01, 0.001]
Iter = 100000
Prec = [1.0e-15, 1.0e-15, 1.0e-15]

Ham = GetHam_IsingCluster(g,J,h)
my_T,my_G = Evo_3site(Ds,Ham,Tau_list,Iter,Prec)

Convergence is achieved!


In [114]:
print("(1) Magnetization")
Cal_mag(my_T,my_G)

print("\n(2) Energy")
Eng = Cal_energy(my_T, my_G, Ham)

print("\n(3) Entanglement")
Cal_entanglement(my_T, my_G)

(1) Magnetization
bond i = 0, <\sigma_x>: 0.4497740171328866, <\sigma_z>: 0.7391289425707998
bond i = 1, <\sigma_x>: 0.44911445420513546, <\sigma_z>: 0.7398519781418951
bond i = 2, <\sigma_x>: 0.44971885477024043, <\sigma_z>: 0.7391590144300675
average <\sigma_x>: 0.44953577536942085
average <\sigma_z>: 0.7393799783809207

(2) Energy
bond i = 0, energy: -1.491042002070756
bond i = 1, energy: -1.4918327863614669
bond i = 2, energy: -1.492590945298813
average energy: -1.4918219112436786

(3) Entanglement
bond i = 0, entanglement entopy: 0.15503835872018487, entanglement spectrum: [0.01764924 1.69404765 3.51996748 5.32201447 5.82359753 6.8896785 ]
bond i = 1, entanglement entopy: 0.15513577081522514, entanglement spectrum: [0.01766313 1.69367871 3.51898026 5.31943709 5.82176432 6.88861417]
bond i = 2, entanglement entopy: 0.15513577081547975, entanglement spectrum: [0.01766313 1.69367871 3.51898026 5.31943709 5.82176432 6.88861417]
average entanglement entropy: 0.15510330011696324, averag